In [ ]:
import os
import time
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.time import Time
from astropy.table import Table, QTable
from ngts.NGTS_image import NGTS_image

In [ ]:
date = "2018-01-19"

base_path = "/Users/XKARAN3135X/Project Data/2018iz"
#drive_path = f"D:/transients/NG0907-1941_{date}" # USB
drive_path = f"/Users/XKARAN3135X/Project Term 2/NGTS Extra Images/NG0504-3633_{date}" # on PC
ref_image_path = base_path + "/NG0504-3633_806_IMAGE80620170816093554_wcs.fits"
ref_catalogue = base_path + "/NG0504-3633_806_IMAGE80620170816093554_cat.fits"

reference_image = NGTS_image(ref_image_path)
# first coordinates are for the star of interest, any subsequent coordinates are for stars to be used for relative photometry
ra = [76.75599, 77.09055, 77.12987, 76.53763, 76.68442, 76.33802]
dec = [-35.92047, -36.15683, -35.681996, -35.99590, -35.67732, -35.67346]
#ra = [76.75599,]
#dec = [-35.92047,]
num_comparison_stars = len(ra)-1

img_files = drive_path
num_images = len(os.listdir(img_files))-2

x, y = reference_image.tweaked_WCS.all_world2pix(ra[0], dec[0], 0)

# data_rows = []
print(num_images)

In [ ]:
# find the optimum photometry aperture using the SNR
apertures = np.arange(0.5, 2, 0.1)
SNRs = []

test_img = os.fsdecode(os.listdir(img_files)[len(os.listdir(img_files))//3]) # pick an image 1/3rd the way through the directory
analysis_img_path = os.path.join(drive_path, test_img)
    
ni = NGTS_image(analysis_img_path)
ni.determine_sky_background()
ni.tweak_wcs(reference_image.tweaked_WCS, ref_catalogue, use_reference_wcs=True)

# perform photometry using multiple aperture sizes
phot = ni.extract_photometry(RA=ra, Dec=dec, apertures=apertures, use_centroids=False)

In [ ]:
# calculate the SNR for each aperture
for radius in apertures:
    r = str(round(radius, 1))
    flux_column = 'FLUX_%s' % r
    err_column = 'FLUX_ERR_%s' % r
    flux = phot[flux_column][0]
    error = phot[err_column][0]
    SNRs.append(flux/error)


plt.plot(apertures, SNRs)
plt.show()

max_SNR_idex = np.argmax(SNRs)
radius = apertures[max_SNR_idex]
r = str(round(radius, 1))
print(radius)  # aperture radius with the largest SNR

In [ ]:
# set up .csv file
csv = f"2018iz_photometry_data_rerun_{date}.csv"
f = open(csv, 'w')
csvtxt = "Time,Flux,Flux_err,Bkg,Bkg_err,"
for j in range(num_comparison_stars):
    csvtxt += ",compare{n}_flux,compare{n}_error".format(n=j+1)  # add in column for comparison star photometry
csvtxt+='\n'
f.write(csvtxt)
f.close()
f = open(csv, 'a')

Dt_list = []

# loop through all images and perform photometry on all objects listed by coordinates in ra and dec
i=1
for file in os.listdir(img_files):
    
    t1 = time.time()
    
    filename = os.fsdecode(file)
    if not filename.startswith("IMAGE"):  # check for image file
        continue
    
    analysis_img_path = os.path.join(drive_path, filename)
    
    ni = NGTS_image(analysis_img_path)
    ni.determine_sky_background()
    ni.tweak_wcs(reference_image.tweaked_WCS, ref_catalogue, use_reference_wcs=True)

    # calculate local times of observation for making light curve
    JD, BJD, HJD, Hltt, Bltt = ni.MJD_to_JD_HJD(RA=ra, Dec=dec, get_light_travel_times=False)

    # perform photometry
    phot = ni.extract_photometry(RA=ra, Dec=dec, apertures=radius, use_centroids=False)

#     data_rows.append((Time(BJD[0], format='jd'), 
#                       phot['FLUX_2.3'][0] * u.ct, phot['FLUX_ERR_2.3'][0] * u.ct,
#                       phot['FLUX_BKG_2.3'][0] * u.ct, phot['FLUX_ERR_BKG_2.3'][0] * u.ct))

    # fill in row of .csv file
    csvtxt = str(BJD[0]) + ',' + str(phot['FLUX_%s' % r][0]) + ',' + str(phot['FLUX_ERR_%s' % r][0]) + ',' + str(phot['FLUX_BKG_%s' % r][0]) + ',' + str(phot['FLUX_ERR_BKG_%s' % r][0]) + ','
    
    print(phot['FLUX_%s' % r][0])
    
    for j in range(num_comparison_stars):
        csvtxt += ',' + str(phot['FLUX_%s' % r][j+1]) + ',' + str(phot['FLUX_ERR_%s' % r][j+1])
    
    csvtxt+='\n'
    f.write(csvtxt)
    
    t2 = time.time()
    Dt_list.append(t2 - t1)
    if len(Dt_list) > 20:
        Dt_list.pop(0)
    average_time_per_image = np.sum(Dt_list) / len(Dt_list)
    time_remaining = average_time_per_image * (num_images-i)
    
    print('analysed ' + str(i) + '/%s images' % num_images)
    print(f'Estimated time remaining: {round(time_remaining/60, 1)} minutes')
    i+=1

f.close()

In [ ]:
# make a plot
fig, ax = ni.plot_image()
ni.plot_apertures(RA=ra, Dec=dec, ax=ax, radius=radius)
ax.set_ylim(y-100, y-100)
ax.set_xlim(x-100, x-100)
#plt.savefig("check.png")
plt.show()

In [ ]:
# timeseries = QTable(rows=data_rows, names=('Time', 'Flux', 'Flux_err', 'Bkg', 'Bkg_err'))
# t = Time(BJD[0], format='jd')
# t.format = 'ymdhms'
# print(t)
# timeseries
f.close()